In [1]:
import cv2
from sklearn.svm import LinearSVC
import pandas as pd
import numpy as np
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier

In [3]:
def extract_histogram(image, bins=(8, 8, 8)):
    hist = cv2.calcHist([image], [0, 1, 2], None, bins, [0, 256, 0, 256, 0, 256])
    cv2.normalize(hist, hist)
    return hist.flatten()

In [10]:
from PIL import Image
import glob
image_list = []
names = []
for filename in glob.glob('train_trees/*.jpg'): #assuming gif
    #names.append(filename)
    if filename[12] == 'c':
        names.append(1)
    else:
        names.append(0)
    im= cv2.imread(filename)
    im = extract_histogram(im)
    image_list.append(im)

In [11]:
svm = LinearSVC(C = 1.09, random_state = 462)

In [12]:
svm.fit(image_list,names)

LinearSVC(C=1.09, random_state=462)

In [23]:
from sklearn.svm import SVC

In [20]:
dc = DecisionTreeClassifier(criterion = 'entropy',min_samples_leaf = 10,max_leaf_nodes = 20,random_state = 462)

In [26]:
bagging = BaggingClassifier(dc, n_estimators=19, random_state=462)


In [27]:
bagging.fit(image_list,names)

BaggingClassifier(base_estimator=DecisionTreeClassifier(criterion='entropy',
                                                        max_leaf_nodes=20,
                                                        min_samples_leaf=10,
                                                        random_state=462),
                  n_estimators=19, random_state=462)

In [28]:
forest = RandomForestClassifier(n_estimators = 19, criterion = 'entropy', min_samples_leaf = 10, max_leaf_nodes = 20, random_state = 462).fit(image_list,names)

In [29]:
log_reg = LogisticRegression(solver='lbfgs', random_state = 462)

In [30]:
base_estimators = [('SVM', svm), ('Bagging DT', bagging), ('DecisionForest', forest)]
sclf = StackingClassifier(estimators=base_estimators, final_estimator=log_reg, cv=2)

In [31]:
sclf.fit(image_list,names)

StackingClassifier(cv=2,
                   estimators=[('SVM', LinearSVC(C=1.09, random_state=462)),
                               ('Bagging DT',
                                BaggingClassifier(base_estimator=DecisionTreeClassifier(criterion='entropy',
                                                                                        max_leaf_nodes=20,
                                                                                        min_samples_leaf=10,
                                                                                        random_state=462),
                                                  n_estimators=19,
                                                  random_state=462)),
                               ('DecisionForest',
                                RandomForestClassifier(criterion='entropy',
                                                       max_leaf_nodes=20,
                                                       min_samples_leaf=10,
       

In [32]:
sclf.score(image_list,names)

0.839

In [40]:
filename = 'test_trees/cat.1004.jpg'
im= cv2.imread(filename)
im = extract_histogram(im)

In [41]:
sclf.predict_proba(im.reshape(1, -1))

array([[0.58395807, 0.41604193]])